In [36]:
from pathlib import Path
import json

# Less Hallucination Analysis

In [37]:
tasks = ["anli", "aqua", "gsm8k", "race-h", "race-m", "strategyqa", "triviaqa", "winogrande"]
models = ["palm-2-text", "palm-2-chat", "gpt-3.5-turbo", "gpt-4-turbo", "llama-2-7b", "llama-2-7b-chat",
          "gemini-pro-text", "gemini-pro-chat"]

In [38]:
def is_not_hallucination(response, model):
    gpt_model = "I'm sorry, but" in response and (
            "Examples" in response or "example" in response or "examples" in response)

    gemini_model_part = "provided context" in response or "provided text" in response or "information" in response
    gemini_model = ("does not mention" in response and gemini_model_part) or (
            "does not contain" in response and gemini_model_part) or (
                           "not available" in response and gemini_model_part) or (
                               "could not extract" in response and gemini_model_part) or (
                               "i cannot answer" in response and gemini_model_part) or (
                               "cannot be found" in response and gemini_model_part)
    
    if model in ["gpt-3.5-turbo", "gpt-4-turbo"]:
        return gpt_model
    if model in ["gemini-pro-text", "gemini-pro-chat"]:
        return gemini_model
    
    return gpt_model or gemini_model


def analyse(task, model):
    model_path = Path('results') / model
    task_path = model_path / task
    null_shot_path = task_path / "null-shot"
    files = sorted(list(null_shot_path.glob("*.json")))
    files = files[:-1]

    output_path = Path("more_analysis") / model
    output_path.mkdir(parents=True, exist_ok=True)

    count = 0
    for idx in range(len(files)):
        file = files[idx]
        with open(file, 'r') as f:
            data = json.load(f)
        if is_not_hallucination(data["response"], model):
            count += 1
            print(f"null_shot: {file.name}")
            with open(output_path / f"{task}-{model}-{idx}.json", 'w') as f:
                json.dump(data, f, indent=4)
    print(f"count: {count}")

In [39]:
for task in tasks:
    for model in models:
        print(f"== task: {task}, model: {model} ==")
        analyse(task, model)

== task: anli, model: palm-2-text ==
count: 0
== task: anli, model: palm-2-chat ==
null_shot: 19.json
null_shot: 607.json
null_shot: 948.json
count: 3
== task: anli, model: gpt-3.5-turbo ==
count: 0
== task: anli, model: gpt-4-turbo ==
count: 0
== task: anli, model: llama-2-7b ==
count: 0
== task: anli, model: llama-2-7b-chat ==
count: 0
== task: anli, model: gemini-pro-text ==
null_shot: 964.json
count: 1
== task: anli, model: gemini-pro-chat ==
count: 0
== task: aqua, model: palm-2-text ==
count: 0
== task: aqua, model: palm-2-chat ==
count: 0
== task: aqua, model: gpt-3.5-turbo ==
count: 0
== task: aqua, model: gpt-4-turbo ==
null_shot: 152.json
count: 1
== task: aqua, model: llama-2-7b ==
count: 0
== task: aqua, model: llama-2-7b-chat ==
count: 0
== task: aqua, model: gemini-pro-text ==
null_shot: 150.json
count: 1
== task: aqua, model: gemini-pro-chat ==
null_shot: 150.json
count: 1
== task: gsm8k, model: palm-2-text ==
count: 0
== task: gsm8k, model: palm-2-chat ==
count: 0
== ta

## Summarization

In [56]:
model_folders = [Path("more_analysis") / model for model in models if (Path("more_analysis") / model).exists()]

In [57]:
filtered_files = []
for folder in model_folders:
    filtered_files.extend([file for file in folder.glob("*.json")])

In [58]:
summary = {}
for file in filtered_files:
    with open(file, 'r') as f:
        data = json.load(f)
    task = data["task"]
    model = data["model"]
    if model not in summary:
        summary[model] = {}
    if task not in summary[model]:
        summary[model][task] = 0
    summary[model][task] += 1

In [59]:
summary

{'gpt-4-turbo': {'strategyqa': 1234,
  'winogrande': 260,
  'triviaqa': 79,
  'race-h': 2,
  'gsm8k': 1,
  'aqua': 1},
 'gemini-pro-text': {'triviaqa': 647,
  'winogrande': 69,
  'strategyqa': 603,
  'gsm8k': 17,
  'race-h': 7,
  'aqua': 1},
 'gemini-pro-chat': {'triviaqa': 648,
  'strategyqa': 601,
  'winogrande': 69,
  'gsm8k': 16,
  'race-h': 8,
  'aqua': 1}}

In [51]:
# Adjusted for false positives
summary["race-h"].pop("llama-2-7b", None)
summary['race-h'].pop('palm-2-chat', None)
summary['winogrande'].pop('palm-2-chat', None)
summary.pop("race-m", None)
summary.pop("anli", None)

In [60]:
summary

{'gpt-4-turbo': {'strategyqa': 1234,
  'winogrande': 260,
  'triviaqa': 79,
  'race-h': 2,
  'gsm8k': 1,
  'aqua': 1},
 'gemini-pro-text': {'triviaqa': 647,
  'winogrande': 69,
  'strategyqa': 603,
  'gsm8k': 17,
  'race-h': 7,
  'aqua': 1},
 'gemini-pro-chat': {'triviaqa': 648,
  'strategyqa': 601,
  'winogrande': 69,
  'gsm8k': 16,
  'race-h': 8,
  'aqua': 1}}

In [61]:
with open("hallucination_summary.json", 'w') as f:
    json.dump(summary, f, indent=4)